In [1]:
%pip install -q "torch>=2.1" "torchvision>=0.16" "ultralytics==8.0.43" onnx --extra-index-url https://download.pytorch.org/whl/cpu

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.49.1 requires protobuf<5.0dev,>=4.21.3, but you have protobuf 3.20.3 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
torchaudio 0.13.0 requires torch==1.13.0, but you have torch 2.2.1 which is incompatible.
yolov5 7.0.13 requires ultralytics>=8.0.100, but you have ultralytics 8.0.43 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from ultralytics import YOLO
from pathlib import Path

In [18]:
DET_MODEL_NAME = "yolov8n"
models_dir = Path("./models")
det_model = YOLO(models_dir / f'{DET_MODEL_NAME}.pt')

In [19]:
# object detection model
det_model_path = models_dir / f"{DET_MODEL_NAME}_openvino_model/{DET_MODEL_NAME}.xml"
if not det_model_path.exists():
    det_model.export(format="openvino", dynamic=True, half=False)

In [20]:
import ipywidgets as widgets
import openvino as ov

core = ov.Core()

device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

Dropdown(description='Device:', index=1, options=('CPU', 'AUTO'), value='AUTO')

In [22]:
core = ov.Core()

det_ov_model = core.read_model(det_model_path)
if device.value != "CPU":
    det_ov_model.reshape({0: [1, 3, 640, 640]})
ov_config = {}
if "GPU" in device.value or ("AUTO" in device.value and "GPU" in core.available_devices):
    ov_config = {"GPU_DISABLE_WINOGRAD_CONVOLUTION": "YES"}
det_compiled_model = core.compile_model(det_ov_model, device.value, ov_config)